[Reference](https://github.com/openai/CLIP/blob/main/notebooks/Prompt_Engineering_for_ImageNet.ipynb)

In [1]:

import numpy as np
import torch
import clip
from tqdm.notebook import tqdm
from pkg_resources import packaging
print(clip.available_models())
model, preprocess = clip.load("ViT-B/32")
CUDA_LAUNCH_BLOCKING=1
model.to('cuda:1')


['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']


CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [2]:
cifar10_classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
# imagenet_templates = [
#     'a bad photo of a {}.',
#     'a photo of many {}.',
#     'a sculpture of a {}.',
#     'a photo of the hard to see {}.',
#     'a low resolution photo of the {}.',
#     'a rendering of a {}.',
#     'graffiti of a {}.',
#     'a bad photo of the {}.',
#     'a cropped photo of the {}.',
#     'a tattoo of a {}.',
#     'the embroidered {}.',
#     'a photo of a hard to see {}.',
#     'a bright photo of a {}.',
#     'a photo of a clean {}.',
#     'a photo of a dirty {}.',
#     'a dark photo of the {}.',
#     'a drawing of a {}.',
#     'a photo of my {}.',
#     'the plastic {}.',
#     'a photo of the cool {}.',
#     'a close-up photo of a {}.',
#     'a black and white photo of the {}.',
#     'a painting of the {}.',
#     'a painting of a {}.',
#     'a pixelated photo of the {}.',
#     'a sculpture of the {}.',
#     'a bright photo of the {}.',
#     'a cropped photo of a {}.',
#     'a plastic {}.',
#     'a photo of the dirty {}.',
#     'a jpeg corrupted photo of a {}.',
#     'a blurry photo of the {}.',
#     'a photo of the {}.',
#     'a good photo of the {}.',
#     'a rendering of the {}.',
#     'a {} in a video game.',
#     'a photo of one {}.',
#     'a doodle of a {}.',
#     'a close-up photo of the {}.',
#     'a photo of a {}.',
#     'the origami {}.',
#     'the {} in a video game.',
#     'a sketch of a {}.',
#     'a doodle of the {}.',
#     'a origami {}.',
#     'a low resolution photo of a {}.',
#     'the toy {}.',
#     'a rendition of the {}.',
#     'a photo of the clean {}.',
#     'a photo of a large {}.',
#     'a rendition of a {}.',
#     'a photo of a nice {}.',
#     'a photo of a weird {}.',
#     'a blurry photo of a {}.',
#     'a cartoon {}.',
#     'art of a {}.',
#     'a sketch of the {}.',
#     'a embroidered {}.',
#     'a pixelated photo of a {}.',
#     'itap of the {}.',
#     'a jpeg corrupted photo of the {}.',
#     'a good photo of a {}.',
#     'a plushie {}.',
#     'a photo of the nice {}.',
#     'a photo of the small {}.',
#     'a photo of the weird {}.',
#     'the cartoon {}.',
#     'art of the {}.',
#     'a drawing of the {}.',
#     'a photo of the large {}.',
#     'a black and white photo of a {}.',
#     'the plushie {}.',
#     'a dark photo of a {}.',
#     'itap of a {}.',
#     'graffiti of the {}.',
#     'a toy {}.',
#     'itap of my {}.',
#     'a photo of a cool {}.',
#     'a photo of a small {}.',
#     'a tattoo of the {}.',
# ]

In [3]:
from torchvision import datasets, transforms

# 定义一个 transform，先将图像转换为 tensor，然后 resize 到 224x224
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# 使用 torchvision.datasets.CIFAR10 来加载 CIFAR-10 数据集
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# 使用 DataLoader 来批量加载数据
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [4]:

# def zeroshot_classifier(classnames, templates):
#     with torch.no_grad():
#         zeroshot_weights = []
#         for classname in tqdm(classnames):
#             texts = [template.format(classname) for template in templates] #format with class
#             texts = clip.tokenize(texts).to('cuda:1') #tokenize
#             class_embeddings = model.encode_text(texts) #embed with text encoder
#             class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
#             class_embedding = class_embeddings.mean(dim=0)
#             class_embedding /= class_embedding.norm()
#             zeroshot_weights.append(class_embedding)
#         zeroshot_weights = torch.stack(zeroshot_weights, dim=1).to('cuda:1')
#     return zeroshot_weights


# zeroshot_weights = zeroshot_classifier(cifar10_classes, imagenet_templates)

In [5]:
text_inputs = torch.cat([clip.tokenize(f"a photo of {c}") for c in cifar10_classes]).to('cuda:1')
top1, top5, n = 0., 0., 0.
with torch.no_grad():
    for i, (images, target) in enumerate(tqdm(trainloader)):
        images = images.to('cuda:1')
        target = target.to('cuda:1')
        image_features = model.encode_image(images)
        text_features = model.encode_text(text_inputs)       
        # Pick the top 5 most similar labels for the image
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
        values, indices = torch.topk(similarity,k=1,dim=1)
        indices = indices.squeeze(1)
        top1 += (indices==target).sum().item()
        n += target.size(0)
print('Accuracy of the network on the test images: {}%'.format((100 * top1 / n)))

  0%|          | 0/50000 [00:00<?, ?it/s]

RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
